In [3]:
import psycopg2 as db
import hashlib

def get_connection():
    conn = db.connect(
        host="localhost",
        database="cloudbox",
        user="postgres",
        password="khanzode"
    )
    print("Connection established")
    return conn

conn = get_connection()
cur = conn.cursor()

Connection established


In [ ]:
def init_db():
    cur.execute("""
    CREATE TABLE IF NOT EXISTS file_metadata (
        file_id SERIAL PRIMARY KEY,
        file_hash TEXT,
        file_name TEXT UNIQUE,
        file_size INTEGER,
        upload_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        is_deleted BOOLEAN DEFAULT FALSE,
        restored_from_trash BOOLEAN DEFAULT FALSE
    )
    """)
    conn.commit() 
    print("Docs table initialized successfully")

In [ ]:
def get_file_hash(file):
    content = file.read()
    file_hash = hashlib.sha256(content).hexdigest()
    file.seek(0)
    return file_hash

def add_entry(file, file_size, file_hash):
    cur.execute("""
        INSERT INTO file_metadata (file_hash, file_name, file_size) VALUES (%s, %s, %d)
    """, file_hash, file.filename, file_size)
    
    print("File metadata updated in DB")

In [ ]:
'''
    0 : file exists and not modified
    1 : file exists but modified
    2 : file doesnt exist
'''

def is_uploaded(file_hash, filename):
    cur.execute("""
        SELECT * FROM file_metadata WHERE file_hash = %s
    """, (file_hash,))
    entry = cur.fetchone()

    if entry is None:
            return "new"
    elif entry[0] == file_hash:
        return "duplicate"
    else:
        return "modified"